# Beyond Hello World, A Computer Vision Example

Note: This is a modified version of Course 1 - Part 4 - Lesson 2 - Notebook from Laurence Moroney's Introduction to Tensorflow for AI, ML, and DL. 

In 01_HelloWorld.ipynb, you created a neural network that figured out the simple rule Y=2x-1 using data (learned behavior). Here, we'll apply deep learning on computer vision problem, where writing rules is much more difficult. Let's take a look at a scenario where we can recognize 10 different types of clothing.

## Import Libraries
Let's start with our import of TensorFlow

In [ ]:
import tensorflow as tf
print(tf.__version__)      # print tensorflow version

## Provide Data

For the data, we will be using Fashion MNIST ( https://github.com/zalandoresearch/fashion-mnist ), consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes/ types of clothes. The dataset is available directly in the tf.keras datasets API. You load it like this:

In [ ]:
mnist = tf.keras.datasets.fashion_mnist

Calling load_data() on this object will give you two sets of two lists, these will be the training and testing values for the graphics that contain the clothing items and their labels.

In [ ]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

Using matplotlib, we'll print a training image and its associated training label. Experiment with different indices in the array. For example, index 42 is a different boot than the one at index 0.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
idx=0  # replace with 42 
print("Training Label: {}".format(training_labels[idx]))
plt.imshow(training_images[idx])
print(training_images[idx])

You'll notice that all of the values in the number are between 0 and 255. If we are training a neural network, for various reasons it's easier if the values are **normalized between 0 and 1**.. 

In [ ]:
training_images  = training_images / 255.0    # normalizing training images
test_images = test_images / 255.0             # normalizing test images

## Define and Compile the Neural Network

Let's now design the model. Things to note: 

**Sequential**: defines a SEQUENCE of layers in the neural network

**Flatten**: takes the square image in the output of print(training_images[idx]) and turns it into a 1 dimensional set.

**Dense**: Adds a layer of neurons

Each layer of neurons need an **activation function** to tell them what to do. There's lots of options, but just use these for now. 

**Relu** effectively means "If X>0 return X, else return 0" -- so what it does it it only passes values 0 or greater to the next layer in the network.

**Softmax** takes a set of values, and effectively picks the biggest one, so, for example, if the output of the last layer looks like [0.1, 0.1, 0.05, 0.1, 9.5, 0.1, 0.05, 0.05, 0.05], it saves you from fishing through it looking for the biggest value, and turns it into [0,0,0,0,1,0,0,0,0] -- The goal is to save a lot of coding!

Once the model is defined, you can compile it with an optimizer and loss function.

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
model.compile(optimizer = tf.train.AdamOptimizer(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Train the Neural Network

Train the model by calling **model.fit**, which fits your training data to your training labels -- i.e. have it figure out the relationship between the training data and its actual labels so in future if you have data that looks like the training data, then it can make a prediction for what that data would look like. 

In [ ]:
model.fit(training_images, training_labels, epochs=5)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.subplot(2, 1, 1)
plt.plot(model.history.history['loss'])
plt.ylabel('Loss')
plt.subplot(2, 1, 2)
plt.plot(model.history.history['acc'],color = '#ffc125')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')

Notice how the loss decreases and the accuracy increases over time. Once training is done-- you should see an accuracy value at the end of the final epoch (~0.89). This tells you that your neural network is about 89% accurate in classifying the training data. I.e., it figured out a pattern match between the image and the labels that worked 89% of the time. 

## Evaluate model using unseen data

We use the 10,000 test images to evaluate the model on unseen data. We can call **model.evaluate**, and pass in the two sets (images and labels), and it will report back the loss for each.

In [ ]:
model.evaluate(test_images, test_labels)

For me, that returned an accuracy of about 0.86. As expected, the model probably would not do as well with *unseen* data as it did with data it was trained on!  

# Exploration Exercises

To explore further, try the exploration exercises.

### Exercise 1: Predicted Label
This code creates a set of classifications for each of the test images, and then prints the first entry in the classifications (for image # 0 in the test set). The output, after you run it is a list of numbers representing the probability that this item is each of the 10 classes.

In [ ]:
import numpy as np  # used for rounding prediction
classifications = model.predict(test_images)        
image_number = 0
print(np.round(classifications[image_number],3))

Print(test_labels[0]) will give you a 9. The neural network is telling us that it's most likely or almost certainly a 9 (an ankle boot).

In [ ]:
print(test_labels[image_number])
plt.imshow(test_images[image_number])

The 10th element on the list is the biggest, and the ankle boot is labelled 9.

###  Exercise 2: Increasing number of neurons (wider network)
Experiment with different values for the dense layer starting with 1024 neurons. What different results do you get for loss, training time etc? Why do you think that's the case?

In [ ]:
import tensorflow as tf
print(tf.__version__)

number_of_neurons = 1024    # set number of neurons here
mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

#### apply larger number of neurons to layer
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(number_of_neurons, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(np.round(classifications[0],3))
print(test_labels[0])

With 1024 Neurons the training takes longer (do more calculations), but is more accurate in this case. That doesn't mean it's always a case of 'more is better' as you can hit the law of diminishing returns very quickly! 

### Exercise 3: Increasing number of layers (deeper network)

Consider the effects of additional layers in the network. Here we add a layer with 256 neurons after the first hidden layer. 
Note: Because the data set is relatively simple, adding additional layer doesn't have a significant impact. 

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

# The network has two hidden layers 
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(np.round(classifications[0],3))
print(test_labels[0])

## Exercise 4: Increasing number of epochs (longer training)

Consider the impact of training for more (or less) epochs. 

Try 15 epochs -- you'll probably get a model with a much better loss than the one with 5
Try 30 epochs -- you might see the loss value stops decreasing, and sometimes increases. This is a side effect of something called 'overfitting' and it's something you need to keep an eye out for when training neural networks. There's no point in wasting your time training if you aren't improving your loss, right! :)

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) , (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=30)   # define number of epoch here

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(np.round(classifications[0],3))
print(test_labels[0])

In [ ]:
# Now let's plot the accuracy and loss through the number of epochs
import matplotlib.pyplot as plt
plt.subplot(2, 1, 1)
plt.plot(model.history.history['loss'])
plt.ylabel('Loss')
plt.subplot(2, 1, 2)
plt.plot(model.history.history['acc'],color = '#ffc125')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')

### Exercise 5: Implement callbacks 

Earlier when you trained for extra epochs you had an issue where your loss might change. Here, callbacks are implemented to stop the training when a certain value of loss or accuracy is reached. If you reach 85% accuracy after 3 epochs, the training would stop regardless of the number of epochs in the training. 

In [ ]:
import tensorflow as tf
print(tf.__version__)

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss')<0.25):
      print("\nReached Loss < 0.25 so cancelling training!")
      self.model.stop_training = True
    if(logs.get('acc')>0.85):
      print("\nReached 85% Accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=100, callbacks=[callbacks])